<center>
    <img src="https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

<h1 align=center><font size = 5>Assignment: Notebook for Peer Assignment</font></h1>


# Introduction

Using this Python notebook you will:

1.  Understand three Chicago datasets
2.  Load the three datasets into three tables in a Db2 database
3.  Execute SQL queries to answer assignment questions




### 1. Socioeconomic Indicators in Chicago

This dataset contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” for each Chicago community area, for the years 2008 – 2012.


### 2. Chicago Public Schools

This dataset shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year. This dataset is provided by the city of Chicago's Data Portal.


### 3. Chicago Crime Data

This dataset reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.


### Store the datasets in database tables

To analyze the data using SQL, it first needs to be stored in the database.

While it is easier to read the dataset into a Pandas dataframe and then PERSIST it into the database as we saw in Week 3 Lab 3, it results in mapping to default datatypes which may not be optimal for SQL querying. For example a long textual field may map to a CLOB instead of a VARCHAR.

Therefore, **it is highly recommended to manually load the table using the database console LOAD tool, as indicated in Week 2 Lab 1 Part II**. The only difference with that lab is that in Step 5 of the instructions you will need to click on create "(+) New Table" and specify the name of the table you want to create and then click "Next".

<img src = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/LoadingData.png">

##### Now open the Db2 console, open the LOAD tool, Select / Drag the .CSV file for the first dataset, Next create a New Table, and then follow the steps on-screen instructions to load the data. Name the new tables as follows:

1.  **CENSUS_DATA**
2.  **CHICAGO_PUBLIC_SCHOOLS**
3.  **CHICAGO_CRIME_DATA**


### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [1]:
!pip install sqlalchemy==1.3.9
!pip install ibm_db_sa

In [2]:
%load_ext sql

In the next cell enter your db2 connection string. Recall you created Service Credentials for your Db2 instance in first lab in Week 3. From your Db2 service credentials copy everything after db2:// (except the double quote at the end) and paste it in the cell below after ibm_db_sa://

<img src ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/images/details.png">


In [3]:
# Remember the connection string is of the format:
# %sql ibm_db_sa://my-username:my-password@my-hostname:my-port/my-db-name?security=SSL
# Enter the connection string for your Db2 on Cloud database instance below
%sql ibm_db_sa://qxs77466:*****@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb?security=SSL

'Connected: qxs77466@bludb'

## Problems

Now write and execute SQL queries to solve assignment problems

### Problem 1

##### Find the total number of crimes recorded in the CRIME table.


In [4]:
%sql select count(*) as "Total Number of Crimes" from crime_data

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


Total Number of Crimes
533


### Problem 2

##### List community areas with per capita income less than 11000.


In [5]:
%sql select community_area_number as "Area Number", \
community_area_name as "Area Name", \
per_capita_income as "Per Capita Income" \
from census_data \
where per_capita_income < 11000

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


Area Number,Area Name,Per Capita Income
26,West Garfield Park,10934
30,South Lawndale,10402
37,Fuller Park,10432
54,Riverdale,8201


### Problem 3

##### List all case numbers for crimes  involving minors?(children are not considered minors for the purposes of crime analysis)


In [6]:
%sql select ID, Case_Number from crime_data \
where description like '%MINOR'

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


id,case_number
3987219,HL266884
3266814,HK238408


### Problem 4

##### List all kidnapping crimes involving a child?


In [7]:
%sql select * from crime_Data where primary_type = 'KIDNAPPING'

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


id,case_number,DATE,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area_number,fbicode,x_coordinate,y_coordinate,YEAR,latitude,longitude,location
5276766,HN144152,2007-01-26,050XX W VAN BUREN ST,1792,KIDNAPPING,CHILD ABDUCTION/STRANGER,STREET,FALSE,FALSE,1533,15,29,25,20,1143050,1897546,2007,41.87490841,-87.75024931,"(41.874908413, -87.750249307)"


### Problem 5

##### What kinds of crimes were recorded at schools?


In [8]:
%sql select distinct(Primary_Type) from crime_data \
where location_description like 'SCHOOL%'

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


primary_type
ASSAULT
BATTERY
CRIMINAL DAMAGE
CRIMINAL TRESPA
NARCOTICS
PUBLIC PEACE VI


### Problem 6

##### List the average safety score for all types of schools.


In [9]:
%sql select "Elementary, Middle, or High School" as "School Type", \
avg(safety_score) as "Average Safety Score"  \
from school_Data \
group by "Elementary, Middle, or High School" 

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


School Type,Average Safety Score
ES,49
HS,49
MS,48


### Problem 7

##### List 5 community areas with highest % of households below poverty line


In [10]:
%sql select Community_area_number, \
Community_area_name, \
percent_households_below_poverty \
from census_data \
order by percent_households_below_poverty desc limit 5 

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


community_area_number,community_area_name,percent_households_below_poverty
54,Riverdale,56.5
37,Fuller Park,51.2
68,Englewood,46.6
29,North Lawndale,43.1
27,East Garfield Park,42.4


### Problem 8

##### Which community area is most crime prone?


In [11]:
table = %sql select community_area_number as "Most Frequent Community Area Number", \
COUNT(community_area_number) as "Number of crimes" \
from crime_Data \
group by community_area_number \
order by count(community_area_number) DESC limit 1; 
print (table)
print ("Most frequent area number is 25, with 43 committed crimes")
%sql select community_area_name from census_data \
where community_area_number = 25

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.
+-------------------------------------+------------------+
| Most Frequent Community Area Number | Number of crimes |
+-------------------------------------+------------------+
|                  25                 |        43        |
+-------------------------------------+------------------+
Most frequent area number is 25, with 43 committed crimes
 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


community_area_name
Austin


Double-click **here** for a hint

<!--
Query for the 'community area number' that is most crime prone.
-->


### Problem 9

##### Use a sub-query to find the name of the community area with highest hardship index


In [12]:
%sql select community_Area_name from census_Data \
where hardship_index =  (select max(hardship_index) from census_data)

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


community_area_name
Riverdale


### Problem 10

##### Use a sub-query to determine the Community Area Name with most number of crimes?


In [13]:
%%sql 
select community_area_name
as "Community Name with Most Number of Crimes "
from census_data 
where  Community_area_number
in (select community_area_number from
(select community_area_number, COUNT(community_area_number)
from crime_Data 
group by community_area_number 
order by count(community_area_number) DESC limit 1))

 * ibm_db_sa://qxs77466:***@54a2f15b-5c0f-46df-8954-7e38e612c2bd.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud:32733/bludb
Done.


Community Name with Most Number of Crimes
Austin


Copyright © 2020 [cognitiveclass.ai](cognitiveclass.ai?utm_source=bducopyrightlink\&utm_medium=dswb\&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01\&cm_mmc=Email_Newsletter-\_-Developer_Ed%2BTech-\_-WW_WW-\_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork-20127838\&cm_mmca1=000026UJ\&cm_mmca2=10006555\&cm_mmca3=M12345678\&cvosrc=email.Newsletter.M12345678\&cvo_campaign=000026UJ).


## Author(s)

<h4> Hima Vasudevan </h4>
<h4> Rav Ahuja </h4>
<h4> Ramesh Sannreddy </h4>

## Contribtuor(s)

<h4> Malika Singla </h4>

## Change log

| Date       | Version | Changed by    | Change Description                 |
| ---------- | ------- | ------------- | ---------------------------------- |
| 2021-05-19 | 2.4     | Lakshmi Holla | Updated the question   |
| 2021-04-30 | 2.3     | Malika Singla | Updated the libraries   |
| 2021-01-15 | 2.2     | Rav Ahuja | Removed problem 11 and fixed changelog |
| 2020-11-25 | 2.1     | Ramesh Sannareddy | Updated the problem statements, and datasets |
| 2020-09-05 | 2.0     | Malika Singla | Moved lab to course repo in GitLab |
| 2018-07-18 | 1.0     | Rav Ahuja | Several updates including loading instructions |
| 2018-05-04 | 0.1     | Hima Vasudevan | Created initial version |

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
